In [19]:
import numpy as np
import tensorflow as tf
import pathlib

In [20]:
data_dir = tf.keras.utils.get_file(
    origin="http://mireu-server.iptime.org:8000/list/HDD1/hana/Garbage_Images.tar.gz",
    fname="Garbage_Images",
    untar=True
)

img_list = pathlib.Path(data_dir)
image_count = len(list(img_list.glob('*/*.jpg')))
images_categories = ['metal', 'glass', 'paper', 'trash', 'cardboard', 'plastic']

In [21]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.1
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(512, 384),
    batch_size=16,
    class_mode='categorical',
    subset='training',
    seed=0
)

validation_generator = test_datagen.flow_from_directory(
    data_dir,
    target_size=(512, 384),
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    seed=0
)

labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

Found 2276 images belonging to 6 classes.
Found 251 images belonging to 6 classes.


In [22]:
def ai_models():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(512, 384, 3)),
        tf.keras.layers.MaxPooling2D(pool_size=2),

        tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=2),

        tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=2),

        tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=2),

        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(64, activation='relu'),

        tf.keras.layers.Dense(len(images_categories), activation='softmax')
    ])
    # model.compile(loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['acc'])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [23]:
# 새로운 모델 객체를 만들고 훈련합니다
model = ai_models()

model.fit(train_generator, validation_data=validation_generator, epochs=50)

# SavedModel로 전체 모델을 저장합니다
model.save('Data/ai_models')

Epoch 1/50


2022-03-30 17:52:54.202998: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 402653184 exceeds 10% of free system memory.
2022-03-30 17:52:57.250343: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 402653184 exceeds 10% of free system memory.


  1/143 [..............................] - ETA: 14:42 - loss: 1.7941 - acc: 0.1875

2022-03-30 17:52:58.564308: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 402653184 exceeds 10% of free system memory.


 47/143 [========>.....................] - ETA: 7:00 - loss: 1.7827 - acc: 0.2194

KeyboardInterrupt: 

In [ ]:
os.listdir(cp_callback)

TypeError: listdir: path should be string, bytes, os.PathLike, integer or None, not ModelCheckpoint

In [ ]:
import matplotlib.pyplot as pyplot

pyplot.figure(figsize=(18, 6))

# 에포크별 정확도
pyplot.subplot(1,2,1)
pyplot.plot(model.history["acc"], label="accuracy")
pyplot.plot(model.history["val_acc"], label="val_accuracy")
pyplot.title("accuracy")
pyplot.legend()

# 에포크별 손실률
pyplot.subplot(1,2,2)
pyplot.plot(model.history["loss"], label="loss")
pyplot.plot(model.history["val_loss"], label="val_loss")
pyplot.title("loss")
pyplot.legend()

pyplot.show()